In [ ]:
import cnoid.CGALUtil as cgal
exec(open('/choreonoid_ws/install/share/irsl_choreonoid/sample/irsl_import.py').read())

light_blue_color = (173/255, 216/255, 230/255)
yellow_color = (1,1,0)
red_color    = (1,0,0)
green_color  = (0, 66/255, 37/255)
black_bean_color = (61/255, 12/255, 2/255)
african_violet_color = (178/255, 132/255, 190/255)
bright_green_color = (102/255, 255/255, 0)

def makeOnEdgeTetra(from_pt, to_pt, cube_size, c_size):
    zz = to_pt - from_pt
    zz = zz/np.linalg.norm(zz)
    yy = to_pt + from_pt
    yy = yy/np.linalg.norm(yy)
    mat = np.column_stack([np.cross(yy, zz), yy, zz])
    cds=coordinates(from_pt, mat)
    #cc=mkshapes.makeTetrahedron(cube_c_size*5, cube_c_size*5, cube_size*2, center_x=cube_c_size*2.5, center_y=0.0, coords=cds)
    cc=cgal.makeTetrahedron(c_size*5, c_size*5, cube_size*2, center_x=c_size*2.5, center_y=0.0, coords=cds)
    cc.translate(npa([0, 0, -cube_size*0.05]))
    cc.translate(npa([-c_size*2.5, 0, 0]))
    cc.translate(npa([0, -c_size*0.5, 0]))
    return cc

def makeCubeWithC(cube_size, horizontal_c_size=None, vertical_c_size=None, color=None, horizontalEdge=True, verticalEdge=False):
    if horizontal_c_size is None:
        horiaontal_c_size = 0.02 * cube_size
    if vertical_c_size is None:
        vertical_c_size = 0.02 * cube_size
    res = cgal.makeBox(cube_size, color=color)
    #res = [ cgal.makeBox(cube_size, color=color) ]
    size_2 = cube_size*0.5
    lst=((1, 1), (-1, 1), (-1, -1), (1, -1))
    if horizontalEdge:
        for l in range(len(lst)):
            from_pt = npa([lst[l-1][0]*size_2, lst[l-1][1]*size_2, size_2])
            to_pt   = npa([lst[l  ][0]*size_2, lst[l  ][1]*size_2, size_2])
            obj=makeOnEdgeTetra(from_pt, to_pt, cube_size, horizontal_c_size)
            cgal.booleanDifference(res, obj)
            #res.append(obj)
        for l in range(len(lst)):
            from_pt = npa([lst[l-1][0]*size_2, lst[l-1][1]*size_2, -size_2])
            to_pt   = npa([lst[l  ][0]*size_2, lst[l  ][1]*size_2, -size_2])
            obj=makeOnEdgeTetra(from_pt, to_pt, cube_size, horizontal_c_size)
            cgal.booleanDifference(res, obj)
            #res.append(obj)
    if verticalEdge:
        for l in range(len(lst)):
            from_pt = npa([lst[l][0]*size_2, lst[l][1]*size_2, -size_2])
            to_pt   = npa([lst[l][0]*size_2, lst[l][1]*size_2,  size_2])
            obj=makeOnEdgeTetra(from_pt, to_pt, cube_size, vertical_c_size)
            cgal.booleanDifference(res, obj)
            #res.append(obj)
    return res

def makeConnectedCubes(lst, color, cube_size=1.0, cube_c_size=0.04):
    res = []
    for l in lst:
        obj = makeCubeWithC(cube_size, cube_c_size, color=color)
        obj.translate( cube_size * npa(l) )
        res.append(obj)
    return res

def makeConnectedCubesScen(lst, fname, color=None, cube_size=1.0, uri=None):
    res = []
    if uri is None:
        uri = fname
    for l in lst:
        obj = mkshapes.loadScene(fname, fileUri=uri, color=color)
        obj.translate( cube_size * npa(l) )
        res.append(obj)
    return res

def makeConnectedCubesPrimitive(lst, color=None, cube_size=1.0):
    res = []
    for l in lst:
        obj = mkshapes.makeBox(cube_size, color=color)
        obj.translate( cube_size * npa(l) )
        res.append(obj)
    return res

#settings=((0, 0, 0), (0, -1, 0), (0,  0, 1), (0, 1, 1))
def createCubes(name, settings, size, c_size, color=None, dirname='', mode=2, URDF=False, meshURLPrefix=''):
    obj = makeCubeWithC(size, horizontal_c_size=c_size, vertical_c_size=c_size,
                        horizontalEdge=True, verticalEdge=True)
    fname = '{}cube_with_c.scen'.format(dirname)
    mkshapes.exportScene(fname, obj.target, exportMesh=True)
    vis = makeConnectedCubesScen(settings, fname, color=color, cube_size=size)
    col = makeConnectedCubesPrimitive(settings, cube_size=size)
    # Builder
    rb=RobotBuilder()
    # visual
    rb.addShapes(vis)
    # collision
    gcol = cutil.SgGroup()
    for c in col:
        gcol.addChild(c.target)
    ### for zero division / calc mass from collision
    lcur=rb.createLinkFromShape(name='base_link', root=True, density=200, collision=gcol, useCollisionForMassparam=True)
    if URDF:
        urdffname='{}{}.urdf'.format(dirname, name)
        rb.exportURDF(urdffname, RobotName=name, UseURDFPrimitiveGeometry=True, UseXacro=False, MeshURLPrefix=meshURLPrefix, MeshFilePrefix=dirname)
    else:
        bodyfname='{}{}.body'.format(dirname, name)
        rb.exportBody(bodyfname, mode=mode, modelName=name)

In [ ]:
import os

CUBE_SIZE = 0.03
CUBE_C_SIZE=0.001
_meshprefix='package://irsl_sim_environments/urdf/puzzle_blocks/'
_URDF=False
_outputdir='/tmp/cnoid/'
#_URDF=True
#_outputdir='/tmp/urdf/'



In [ ]:
def loadRobot(file):
    ritm=ib.loadRobotItem(file)
    return RobotModel(ritm)

In [ ]:
def deleteRobot():
    w=ib.getOrAddWorld()
    w.removeFromParentItem()

In [ ]:
def getvertex_lists(settings):
    cube_vertexs = [
                [0,0,0],
                [0,1,0],
                [1,1,0],
                [1,0,0],
                [0,0,1],
                [0,1,1],
                [1,1,1],
                [1,0,1],
             ]
    vertex_map = {}
    for setting in settings:
        for vertex in cube_vertexs:
            tmp = list(setting)
            for i in range(3):
                tmp[i] += vertex[i]
            if tuple(tmp) not in vertex_map:
                vertex_map[tuple(tmp)]=[setting]
            else :
                vertex_map[tuple(tmp)].append(setting)
    
    v_list = []
    cv_list = []
    for v, cube_list in vertex_map.items():
        n = len(cube_list)
        v_type = "None"
        if n == 1:
            v_type = "vertex"
        elif n in [3,5,7]:
            v_type = "c-vertex"
        elif n == 4:
            flag = True
            for i in range(3):
                tmp = len(set([c[i] for c in cube_list]))
                if tmp == 1:
                    flag = False
                    break
            if flag :
                v_type = "c-vertex"
        # print(v, n, v_type)
        if v_type == 'vertex':
            v_list.append((fv(*v)-0.5)*CUBE_SIZE)
        elif v_type == 'c-vertex':
            cv_list.append((fv(*v)-0.5)*CUBE_SIZE)
    return v_list, cv_list


In [ ]:
import random
def setRandomCamera():
    theta = random.gauss(0, PI/2)
    elv   = random.gauss(0, PI/4)
    ar = IC.normalizeVector(fv(math.sin(theta), math.cos(theta), math.tan(elv)))
    #ln = mkshapes.makeLines([[0, 0, 0], ar.tolist()])
    #di.addObject(ln)
    cds=ib.cameraPositionLookingAt(ar, fv(0, 0, 0), fv(0, 0, 1))
    ib.setCameraCoords(cds)
    ib.viewAll()

In [ ]:
def setEnvironment(size=512, color=[0, 0, 0], **kwargs):
    ib.setViewSize(size, size)
    ib.setBackgroundColor(color)
    ib.disableGrid()
    ib.setCoordinateAxes(False)


In [ ]:
def saveImage(prefix, *args):
    ib.viewAll()
    filename = prefix.format(*args)
    ib.saveImageOfScene(filename)

In [ ]:
import cv2
def make_visible_list(pos_list, proj_list):
    image_file = '/tmp/temp.png'
    ret_list = []
    for idx in range(len(pos_list)):
        p = mkshapes.makePoints(pos_list[idx:idx+1], pointSize=5.0)
        di.addObject(p)
        saveImage(image_file)
        tmp_img = cv2.imread(image_file)
        p_x = int(proj_list[idx][0])
        p_y = int(proj_list[idx][1])
        if p_x >= 0 and p_x < tmp_img.shape[1] and p_y >= 0 and p_y < tmp_img.shape[0]:
            p_color = tmp_img[p_y, p_x]
            ret_list.append((p_color>224).all())
        else :
            ret_list.append(False)
        di.clear()
    return ret_list


In [ ]:
block_settings = {'yellow_block':{'color':yellow_color, 'settings': ((0, 0, 0), (0, 0, 1), (0, 1, 1), (0, 2, 1))},
                'cyan_block':{'color':light_blue_color, 'settings': ((0, 0, 0), (0, -1, 1), (0,  0, 1), (0, 1, 1))},
                'red_block':{'color':red_color, 'settings': ((0, 0, 0), (0, -1, 0), (0, 0, 1), (0, 1, 1))},
                'green_block':{'color':green_color, 'settings': ((0, 0, 0), (1, 0, 1), (0, 0, 1), (0, 1, 1))},
                'brown_block':{'color':black_bean_color, 'settings': ((0, 0, 0), (1, 0, 0), (0, 0, 1), (1, 0, 1))},
                'purple_block':{'color':african_violet_color, 'settings': ((0, 0, 0), (0, 0, 1), (1, 0, 1), (1, -1, 1))},
                'lightgreen_block':{'color':bright_green_color, 'settings': ((0, 0, 0), (0, 0, 1), (0, -1, 1))},
               }

In [ ]:
setEnvironment()


In [ ]:
di = DrawInterface()

In [ ]:
import yaml
dataset_output_dirs = ['/tmp/dataset/train', '/tmp/dataset/test']
for dataset_output_dir, data_per_block in zip(dataset_output_dirs, [250,25]):
    os.makedirs(dataset_output_dir, exist_ok=True)
    image_count = 0
    annos = []
    blocks = []
    for block_name, block_setting in block_settings.items():
        settings=block_setting['settings']
        name=block_name
        _col=block_setting['color']
        os.makedirs('{}{}/'.format(_outputdir, name), exist_ok=True)
        createCubes(name, settings, CUBE_SIZE, CUBE_C_SIZE, color=_col, dirname='{}{}/'.format(_outputdir, name),
                    URDF=_URDF, meshURLPrefix='{}{}/'.format(_meshprefix, name))
        robot = loadRobot('/tmp/cnoid/{}/{}.body'.format(block_name,block_name))
        v_list, cv_list = getvertex_lists(settings)
        blocks.append({"name": block_name,
                      "convex_vertex_pos": np.array(v_list).reshape(-1).tolist(),
                      "concave_vertex_pos": np.array(cv_list).reshape(-1).tolist(),
                      })
        di.clear()
        for i in range(data_per_block):
            setRandomCamera()
            rel_filename = "image_{0:06d}.png".format(image_count)
            saveImage(os.path.join(dataset_output_dir, rel_filename))
            proj_v_list = ib.projectPoints(np.array(v_list))
            proj_cv_list = ib.projectPoints(np.array(cv_list))
            v_visible_list = make_visible_list(v_list, proj_v_list)
            cv_visible_list = make_visible_list(cv_list, proj_cv_list)
            # print(proj_v_list, v_visible_list, proj_cv_list, cv_visible_list)
            anno = {"imagefile": rel_filename,
                    "convex_vertex" : [ [float(pos[0]), float(pos[1]), 1 if vis else 0] for pos, vis in zip (proj_v_list, v_visible_list)],
                    "concave_vertex" : [ [float(pos[0]), float(pos[1]), 1 if vis else 0] for pos, vis in zip (proj_cv_list, cv_visible_list)],
                    "camera_position": [float(d) for d in ib.getCameraCoords()[0].pos], 
                    "camera_orientation": [float(d) for d in ib.getCameraCoords()[0].quaternion],
                    "camera_fov" : float(ib.getCameraCoords()[1]),
                    "camera_matrix" : [float(d) for d in ib.getCameraMatrix().reshape(-1)],
                    "block_name" : block_name,
                   }
            annos.append(anno)
            image_count += 1
        deleteRobot()
    with open(os.path.join(dataset_output_dir, "annotation.yaml"), 'w') as f:
        yaml.dump({"annotations":annos, "blocks":blocks}, f)
